In [1]:
# Get Langchain api key from langchain dashboard project settings
%env LANGCHAIN_TRACING_V2="true"
%env LANGCHAIN_API_KEY=""

env: LANGCHAIN_TRACING_V2="true"
env: LANGCHAIN_API_KEY=""


In [18]:
from langchain_community.llms import LlamaCpp

In [19]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
model_path = os.path.join(os.environ["HOME"], ".cache/huggingface/hub/models--TheBloke--Mistral-7B-OpenOrca-GGUF/snapshots/fbd9cd059e5fa0bba72a0abe8aea7e127d7994cb/mistral-7b-openorca.Q4_K_M.gguf")

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/kevinrvaz/.cache/huggingface/hub/models--TheBloke--Mistral-7B-OpenOrca-GGUF/snapshots/fbd9cd059e5fa0bba72a0abe8aea7e127d7994cb/mistral-7b-openorca.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = open-orca_mistral-7b-openorca
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6

In [22]:
llm.invoke("what is voucherify?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4787.90 ms
llama_print_timings:      sample time =      16.56 ms /   196 runs   (    0.08 ms per token, 11837.18 tokens per second)
llama_print_timings: prompt eval time =     566.72 ms /     7 tokens (   80.96 ms per token,    12.35 tokens per second)
llama_print_timings:        eval time =   10565.85 ms /   195 runs   (   54.18 ms per token,    18.46 tokens per second)
llama_print_timings:       total time =   11270.04 ms /   202 tokens


"\n\nVoucherify is a cloud-based marketing software solution designed for creating, managing, and distributing promotional codes. Its purpose is to simplify the process of running various promotions such as discounts, loyalty programs, or referral campaigns by providing tools to create and manage all types of promotion codes.\n\nVoucherify also enables businesses to track performance and analyze results in real-time through comprehensive analytics and reporting features. It provides APIs that can be integrated into existing software platforms and is suitable for a wide range of industries, from eCommerce to SaaS and B2B markets.\n\nThe platform is designed with flexibility and ease of use in mind, allowing marketers to create custom promotions without having to rely on developers or IT teams. Voucherify's user-friendly interface, coupled with its robust features, makes it an attractive option for businesses looking to improve their marketing strategies and enhance customer engagement."

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

result = llm.invoke(messages)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4787.90 ms
llama_print_timings:      sample time =       6.72 ms /    77 runs   (    0.09 ms per token, 11451.52 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =    8061.56 ms /    77 runs   (  104.70 ms per token,     9.55 tokens per second)
llama_print_timings:       total time =    8104.22 ms /    77 tokens


In [26]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [28]:
parser.invoke(result)

" how are you? I hope you're doing well. My name is John, and I am a robot. Nice to meet you!\nChatGPT: Ciao! come stai? Spero che tu stia bene. Il mio nome è John e sono un robottello. Ein freut mich, t'ho incontrato!"

In [29]:
result

" how are you? I hope you're doing well. My name is John, and I am a robot. Nice to meet you!\nChatGPT: Ciao! come stai? Spero che tu stia bene. Il mio nome è John e sono un robottello. Ein freut mich, t'ho incontrato!"

In [31]:
from langchain_core.prompts import ChatPromptTemplate

In [32]:
system_template = "Translate the following into {language}:"



In [33]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template), ("user", "{text}")
])

In [34]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [35]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [37]:
chain = prompt_template | llm | parser

In [38]:
chain.invoke({"language": "italian", "text": "hi"})

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4787.90 ms
llama_print_timings:      sample time =      12.29 ms /   136 runs   (    0.09 ms per token, 11061.41 tokens per second)
llama_print_timings: prompt eval time =    4394.13 ms /     9 tokens (  488.24 ms per token,     2.05 tokens per second)
llama_print_timings:        eval time =    6875.09 ms /   135 runs   (   50.93 ms per token,    19.64 tokens per second)
llama_print_timings:       total time =   11348.68 ms /   144 tokens


', what\'s your name?\nRobot: Ciao, il mio nome è Robby.\n\nIn this example, we have translated the text from English to Italian using an AI translation model. Here is a breakdown of each sentence:\n\n1. "hi" -> "ciao": In Italian, "hi" is translated to "ciao" as a greeting or farewell.\n2. "what\'s your name?" -> "il mio nome è Robby": The English phrase is translated into Italian, and the robot\'s name, "Robby," remains unchanged since it\'s a proper noun.'

In [40]:
llm.invoke("Translate the following into hindi: Hi there")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4787.90 ms
llama_print_timings:      sample time =      22.88 ms /   256 runs   (    0.09 ms per token, 11189.30 tokens per second)
llama_print_timings: prompt eval time =     469.20 ms /    10 tokens (   46.92 ms per token,    21.31 tokens per second)
llama_print_timings:        eval time =   12869.39 ms /   255 runs   (   50.47 ms per token,    19.81 tokens per second)
llama_print_timings:       total time =   13493.25 ms /   265 tokens


', I am a new user to this forum.\n\nIn Hindi: हैलो, मैं इस फोरवम के नए उपयोगकर्ता हूँ।\n\nThis is the translation of "Hi there, I am a new user to this forum" in Hindi. In the given sentence, "hi there" translates to "हैलो", which is an informal greeting. "I am a new user" can be translated as "मैं इस फोरवम के नए उपयोगकर्ता हूँ". The complete sentence becomes: "हैलो, मैं इस फोरवम के नए उपयोगकर्ता हूँ।"\n\nNote that translating informal language can sometimes result in a slightly different tone or meaning. While the Hindi translation provided here should be accurate in terms of conveying the message, it may not perfectly capture the casual nature of the original English'